# Load modules

In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

## imports
# general
import json
import pathlib
from copy import deepcopy
import itertools
from itertools import chain
import math
from functools import reduce

# data science
import numpy as np
import pandas as pd
# from scipy.stats import pearsonr
import scipy.cluster.hierarchy as sch
# from sklearn.linear_model import LinearRegression
import scikit_posthocs as sp # post-hoc tests for statistical analysis
from scipy.stats import pearsonr, spearmanr
import matplotlib as mpl  # main package for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# single cell
import anndata as ad  # main package for single cell data
import scanpy as sc  # main package for single cell analysis
import muon as mu  # package for multi-omic single cell data
import scirpy as ir # package for single cell receptor analysis

# helpers
import os
import sys
sys.path.append('/home/divy/projects/jones-salim_tuning/thresholdpy')
import raval_preprocessing as rpp
import thresholdpy as tp
from jones_helper_functions import *

## settings
# set paths
PATH_PROJ_DIR = pathlib.Path(".")  # project directory
PATH_DATA_DIR = pathlib.Path(PATH_PROJ_DIR) / "data"
PATH_FIG_DIR = pathlib.Path(PATH_PROJ_DIR) / "figures" / "TCR_extraction"
PATH_EXPORT_DIR = pathlib.Path(PATH_PROJ_DIR) / "export" / "TCR_extraction"  # export directory
# create figures directory if it does not exist
PATH_FIG_DIR.mkdir(parents=True, exist_ok=True)
PATH_EXPORT_DIR.mkdir(parents=True, exist_ok=True)

FIG_TYPE = "svg"
DPI_SCALE = 300  # dpi scale for figures
FONT_SIZE = 10  # set font size for figures
sc.settings._file_format_figs = FIG_TYPE
# set scanpy settings
sc.settings.figdir = PATH_FIG_DIR  # set figure directory
# svg format, dpi, transparent background
sc.settings.set_figure_params(format=FIG_TYPE, dpi=DPI_SCALE, transparent=True, fontsize=FONT_SIZE)

# export matplotlib as svgs
# Set the global default figure size
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams["pdf.fonttype"] = 42  # set font type for pdfs
plt.rcParams["ps.fonttype"] = 42  # set font type for postscript files
# plt.rcParams['figure.dpi'] = 600
# plt.rcParams['savefig.dpi'] = 600
# disable grid
mpl.rcParams["axes.grid"] = False  # disable grid in plots
plt.rcParams.update({"font.size": FONT_SIZE})  # set font size for matplotlib


# set random seed for reproducibility
SEED = 82
np.random.seed(SEED)  # set random seed for numpy

/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random_number = np.random.rand()
print(random_number)

0.27522566980133767


## Reloads

In [3]:
import importlib

# importlib.reload(tp)
importlib.reload(sc)
# from scanpy.tools import _score_genes as sc_score_genes

# importlib.reload(sc_score_genes)

<module 'scanpy' from '/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/scanpy/__init__.py'>

# TCR sequence extraction

In [4]:
# mdata_tcr_annotated = mu.read_h5mu(
#     PATH_DATA_DIR / "jones_salim_tcr_annotated.h5mu"
# )

# mdata_tcr_annotated["prot"]

### Load TCR data

In [5]:
# Load the TCR data
adata_tcr = ir.io.read_10x_vdj(PATH_DATA_DIR / f"filtered_contig_annotations.csv")
adata_tcr

/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/anndata/utils.py:354: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)


AnnData object with n_obs × n_vars = 2636 × 0
    uns: 'scirpy_version'
    obsm: 'airr'

In [7]:
mdata_annotated = mu.read_h5mu(
    PATH_DATA_DIR / f"mdata_analyzed.h5mu",
)
mdata_annotated

/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 5684 × 36738
  var:	'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
  2 modalities
    rna:	5684 x 36601
      obs:	'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'outliers_empty', 'outliers_multiplets', 'outliers_mito', 'outliers_ribo', 'S_score', 'G2M_score', 'phase', 'leiden_res1', 'celltypist_leiden_res1', 'celltypist_leiden_res1_conf', 'union_cd4_score', 'union_cd8_score', 'naive-cm-like_score', 'cd45ro+effector-memory_score', 'interferon-signaling_score', 'memory/proliferation_score', 'nk-like/cd45ra+effector-memory_score', 'exhaustion_score', 'cell_type', 'leidenP_res0.7', 'CD4+CTL_score', 'CD4+PD1_score', 'CD4+T_score', 'CD4+Tmem_score', 'CD8+T_score', 'CD8+Tmem_score', 'gdT_score', 'CD8aa(I)_score', 'CD8aa(II)_score', 'NK_score', 'ILC3_score', 'NKT_score', 'T(agonist)_score', 'Tfh_score', 'Th17_score', 'Treg_score', 'Treg(diff)_score', 'cell_type_biorxiv', 'cell_type_ross1', 'cell_type_ross2', 'hs-ImCD4_score', 'hs-MatCD4_score', 'hs-ImCD8_score', 'hs-MatCD8_score', 'hs-CD8_score', 'Naive-/CM-like_score', 'Tem_score', 'TTCRC4_score', 'ISG_score', 'Tmem/prolif_score', 'NKL/Temra_score', 'Exhaustion_score', 'union_cd4_sc_log1p_score', 'union_cd8_sc_log1p_score', 'cd4_sc_log1p_score', 'cd8_sc_log1p_score', 'Naive-/CM-like_sc_log1p_score', 'Tem_sc_log1p_score', 'ISG_sc_log1p_score', 'Tmem/prolif_sc_log1p_score', 'NKL/Temra_sc_log1p_score', 'Exhaustion_sc_log1p_score', 'hs-ImCD4_sc_log1p_score', 'hs-MatCD4_sc_log1p_score', 'hs-ImCD8_sc_log1p_score', 'hs-MatCD8_sc_log1p_score', 'hs-CD8_sc_log1p_score', 'CD4+CTL_sc_log1p_score', 'CD4+PD1_sc_log1p_score', 'CD4+T_sc_log1p_score', 'CD4+Tmem_sc_log1p_score', 'CD8+T_sc_log1p_score', 'CD8+Tmem_sc_log1p_score', 'gdT_sc_log1p_score', 'CD8aa(I)_sc_log1p_score', 'CD8aa(II)_sc_log1p_score', 'NK_sc_log1p_score', 'ILC3_sc_log1p_score', 'NKT_sc_log1p_score', 'T(agonist)_sc_log1p_score', 'Tfh_sc_log1p_score', 'Th17_sc_log1p_score', 'Treg_sc_log1p_score', 'Treg(diff)_sc_log1p_score', 'union_cd4_sc_log1p_zscore_score', 'union_cd8_sc_log1p_zscore_score', 'cd4_sc_log1p_zscore_score', 'cd8_sc_log1p_zscore_score', 'Naive-/CM-like_sc_log1p_zscore_score', 'Tem_sc_log1p_zscore_score', 'ISG_sc_log1p_zscore_score', 'Tmem/prolif_sc_log1p_zscore_score', 'NKL/Temra_sc_log1p_zscore_score', 'Exhaustion_sc_log1p_zscore_score', 'hs-ImCD4_sc_log1p_zscore_score', 'hs-MatCD4_sc_log1p_zscore_score', 'hs-ImCD8_sc_log1p_zscore_score', 'hs-MatCD8_sc_log1p_zscore_score', 'hs-CD8_sc_log1p_zscore_score', 'CD4+CTL_sc_log1p_zscore_score', 'CD4+PD1_sc_log1p_zscore_score', 'CD4+T_sc_log1p_zscore_score', 'CD4+Tmem_sc_log1p_zscore_score', 'CD8+T_sc_log1p_zscore_score', 'CD8+Tmem_sc_log1p_zscore_score', 'gdT_sc_log1p_zscore_score', 'CD8aa(I)_sc_log1p_zscore_score', 'CD8aa(II)_sc_log1p_zscore_score', 'NK_sc_log1p_zscore_score', 'ILC3_sc_log1p_zscore_score', 'NKT_sc_log1p_zscore_score', 'T(agonist)_sc_log1p_zscore_score', 'Tfh_sc_log1p_zscore_score', 'Th17_sc_log1p_zscore_score', 'Treg_sc_log1p_zscore_score', 'Treg(diff)_sc_log1p_zscore_score', 'union_cd4_sc_ccreg_score', 'union_cd8_sc_ccreg_score', 'cd4_sc_ccreg_score', 'cd8_sc_ccreg_score', 'Naive-/CM-like_sc_ccreg_score', 'Tem_sc_ccreg_score', 'ISG_sc_ccreg_score', 'Tmem/prolif_sc_ccreg_score', 'NKL/Temra_sc_ccreg_score', 'Exhaustion_sc_ccreg_score', 'hs-ImCD4_sc_ccreg_score', 'hs-MatCD4_sc_ccreg_score', 'hs-ImCD8_sc_ccreg_score', 'hs-MatCD8_sc_ccreg_score', 'hs-CD8_sc_ccreg_score', 'CD4+CTL_sc_ccreg_score', 'CD4+PD1_sc_ccreg_score', 'CD4+T_sc_ccreg_score', 'CD4+Tmem_sc_ccreg_score', 'CD8+T_sc_ccreg_score', 'CD8+Tmem_sc_ccreg_score', 'gdT_sc_ccreg_score', 'CD8aa(I)_sc_ccreg_score', 'CD8aa(II)_sc_ccreg_score', 'NK_sc_ccreg_score', 'ILC3_sc_ccreg_score', 'NKT_sc_ccreg_score', 'T(agonist)_sc_ccreg_score', 'Tfh_sc_ccreg_score', 'Th17_sc_ccreg_score', 'Treg_sc_ccreg_score', 'Treg(diff)_sc_ccreg_score', 'union_cd4_sc-all_log

### Add TCR data to mdata

In [8]:
# add adata_tcr to mdata_annotated MuData object
mdata = mu.MuData(
    {"rna": mdata_annotated["rna"], "prot": mdata_annotated["prot"], "airr": adata_tcr}
)
mu.pp.intersect_obs(mdata)
mdata

/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)
/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1461: FutureWarnin

MuData object with n_obs × n_vars = 2401 × 36738
  3 modalities
    rna:	2401 x 36601
      obs:	'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'outliers_empty', 'outliers_multiplets', 'outliers_mito', 'outliers_ribo', 'S_score', 'G2M_score', 'phase', 'leiden_res1', 'celltypist_leiden_res1', 'celltypist_leiden_res1_conf', 'union_cd4_score', 'union_cd8_score', 'naive-cm-like_score', 'cd45ro+effector-memory_score', 'interferon-signaling_score', 'memory/proliferation_score', 'nk-like/cd45ra+effector-memory_score', 'exhaustion_score', 'cell_type', 'leidenP_res0.7', 'CD4+CTL_score', 'CD4+PD1_score', 'CD4+T_score', 'CD4+Tmem_score', 'CD8+T_score', 'CD8+Tmem_score', 'gdT_score', 'CD8aa(I)_score', 'CD8aa(II)_score', 'NK_score', 'ILC3_score', 'NKT_score', 'T(agonist)_score', 'Tfh_score', 'Th17_score', 'Treg_score', 'Treg(diff)_score', 'cell_type_biorxiv', 'cell_type_ross1', 'cell_type_ross2', 'hs-ImCD4_score', 'hs-MatCD4_score', 'hs-ImCD8_score', 'hs-MatCD8_score', 'hs-CD8_score', 'Naive-/CM-like_score', 'Tem_score', 'TTCRC4_score', 'ISG_score', 'Tmem/prolif_score', 'NKL/Temra_score', 'Exhaustion_score', 'union_cd4_sc_log1p_score', 'union_cd8_sc_log1p_score', 'cd4_sc_log1p_score', 'cd8_sc_log1p_score', 'Naive-/CM-like_sc_log1p_score', 'Tem_sc_log1p_score', 'ISG_sc_log1p_score', 'Tmem/prolif_sc_log1p_score', 'NKL/Temra_sc_log1p_score', 'Exhaustion_sc_log1p_score', 'hs-ImCD4_sc_log1p_score', 'hs-MatCD4_sc_log1p_score', 'hs-ImCD8_sc_log1p_score', 'hs-MatCD8_sc_log1p_score', 'hs-CD8_sc_log1p_score', 'CD4+CTL_sc_log1p_score', 'CD4+PD1_sc_log1p_score', 'CD4+T_sc_log1p_score', 'CD4+Tmem_sc_log1p_score', 'CD8+T_sc_log1p_score', 'CD8+Tmem_sc_log1p_score', 'gdT_sc_log1p_score', 'CD8aa(I)_sc_log1p_score', 'CD8aa(II)_sc_log1p_score', 'NK_sc_log1p_score', 'ILC3_sc_log1p_score', 'NKT_sc_log1p_score', 'T(agonist)_sc_log1p_score', 'Tfh_sc_log1p_score', 'Th17_sc_log1p_score', 'Treg_sc_log1p_score', 'Treg(diff)_sc_log1p_score', 'union_cd4_sc_log1p_zscore_score', 'union_cd8_sc_log1p_zscore_score', 'cd4_sc_log1p_zscore_score', 'cd8_sc_log1p_zscore_score', 'Naive-/CM-like_sc_log1p_zscore_score', 'Tem_sc_log1p_zscore_score', 'ISG_sc_log1p_zscore_score', 'Tmem/prolif_sc_log1p_zscore_score', 'NKL/Temra_sc_log1p_zscore_score', 'Exhaustion_sc_log1p_zscore_score', 'hs-ImCD4_sc_log1p_zscore_score', 'hs-MatCD4_sc_log1p_zscore_score', 'hs-ImCD8_sc_log1p_zscore_score', 'hs-MatCD8_sc_log1p_zscore_score', 'hs-CD8_sc_log1p_zscore_score', 'CD4+CTL_sc_log1p_zscore_score', 'CD4+PD1_sc_log1p_zscore_score', 'CD4+T_sc_log1p_zscore_score', 'CD4+Tmem_sc_log1p_zscore_score', 'CD8+T_sc_log1p_zscore_score', 'CD8+Tmem_sc_log1p_zscore_score', 'gdT_sc_log1p_zscore_score', 'CD8aa(I)_sc_log1p_zscore_score', 'CD8aa(II)_sc_log1p_zscore_score', 'NK_sc_log1p_zscore_score', 'ILC3_sc_log1p_zscore_score', 'NKT_sc_log1p_zscore_score', 'T(agonist)_sc_log1p_zscore_score', 'Tfh_sc_log1p_zscore_score', 'Th17_sc_log1p_zscore_score', 'Treg_sc_log1p_zscore_score', 'Treg(diff)_sc_log1p_zscore_score', 'union_cd4_sc_ccreg_score', 'union_cd8_sc_ccreg_score', 'cd4_sc_ccreg_score', 'cd8_sc_ccreg_score', 'Naive-/CM-like_sc_ccreg_score', 'Tem_sc_ccreg_score', 'ISG_sc_ccreg_score', 'Tmem/prolif_sc_ccreg_score', 'NKL/Temra_sc_ccreg_score', 'Exhaustion_sc_ccreg_score', 'hs-ImCD4_sc_ccreg_score', 'hs-MatCD4_sc_ccreg_score', 'hs-ImCD8_sc_ccreg_score', 'hs-MatCD8_sc_ccreg_score', 'hs-CD8_sc_ccreg_score', 'CD4+CTL_sc_ccreg_score', 'CD4+PD1_sc_ccreg_score', 'CD4+T_sc_ccreg_score', 'CD4+Tmem_sc_ccreg_score', 'CD8+T_sc_ccreg_score', 'CD8+Tmem_sc_ccreg_score', 'gdT_sc_ccreg_score', 'CD8aa(I)_sc_ccreg_score', 'CD8aa(II)_sc_ccreg_score', 'NK_sc_ccreg_score', 'ILC3_sc_ccreg_score', 'NKT_sc_ccreg_score', 'T(agonist)_sc_ccreg_score', 'Tfh_sc_ccreg_score', 'Th17_sc_ccreg_score', 'Treg_sc_ccreg_score', 'Treg(diff)_sc_ccreg_score', 'union_cd4_sc-all_log1p_score', 'union_cd8_sc-all_log1p_score', 'cd4_sc-all_log1p_score', 'cd8_sc

In [9]:
ir.pp.index_chains(mdata)
ir.tl.chain_qc(mdata)
mdata

MuData object with n_obs × n_vars = 2401 × 36738
  3 modalities
    rna:	2401 x 36601
      obs:	'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'outliers_empty', 'outliers_multiplets', 'outliers_mito', 'outliers_ribo', 'S_score', 'G2M_score', 'phase', 'leiden_res1', 'celltypist_leiden_res1', 'celltypist_leiden_res1_conf', 'union_cd4_score', 'union_cd8_score', 'naive-cm-like_score', 'cd45ro+effector-memory_score', 'interferon-signaling_score', 'memory/proliferation_score', 'nk-like/cd45ra+effector-memory_score', 'exhaustion_score', 'cell_type', 'leidenP_res0.7', 'CD4+CTL_score', 'CD4+PD1_score', 'CD4+T_score', 'CD4+Tmem_score', 'CD8+T_score', 'CD8+Tmem_score', 'gdT_score', 'CD8aa(I)_score', 'CD8aa(II)_score', 'NK_score', 'ILC3_score', 'NKT_score', 'T(agonist)_score', 'Tfh_score', 'Th17_score', 'Treg_score', 'Treg(diff)_score', 'cell_type_biorxiv', 'cell_type_ross1', 'cell_type_ross2', 'hs-ImCD4_score', 'hs-MatCD4_score', 'hs-ImCD8_score', 'hs-MatCD8_score', 'hs-CD8_score', 'Naive-/CM-like_score', 'Tem_score', 'TTCRC4_score', 'ISG_score', 'Tmem/prolif_score', 'NKL/Temra_score', 'Exhaustion_score', 'union_cd4_sc_log1p_score', 'union_cd8_sc_log1p_score', 'cd4_sc_log1p_score', 'cd8_sc_log1p_score', 'Naive-/CM-like_sc_log1p_score', 'Tem_sc_log1p_score', 'ISG_sc_log1p_score', 'Tmem/prolif_sc_log1p_score', 'NKL/Temra_sc_log1p_score', 'Exhaustion_sc_log1p_score', 'hs-ImCD4_sc_log1p_score', 'hs-MatCD4_sc_log1p_score', 'hs-ImCD8_sc_log1p_score', 'hs-MatCD8_sc_log1p_score', 'hs-CD8_sc_log1p_score', 'CD4+CTL_sc_log1p_score', 'CD4+PD1_sc_log1p_score', 'CD4+T_sc_log1p_score', 'CD4+Tmem_sc_log1p_score', 'CD8+T_sc_log1p_score', 'CD8+Tmem_sc_log1p_score', 'gdT_sc_log1p_score', 'CD8aa(I)_sc_log1p_score', 'CD8aa(II)_sc_log1p_score', 'NK_sc_log1p_score', 'ILC3_sc_log1p_score', 'NKT_sc_log1p_score', 'T(agonist)_sc_log1p_score', 'Tfh_sc_log1p_score', 'Th17_sc_log1p_score', 'Treg_sc_log1p_score', 'Treg(diff)_sc_log1p_score', 'union_cd4_sc_log1p_zscore_score', 'union_cd8_sc_log1p_zscore_score', 'cd4_sc_log1p_zscore_score', 'cd8_sc_log1p_zscore_score', 'Naive-/CM-like_sc_log1p_zscore_score', 'Tem_sc_log1p_zscore_score', 'ISG_sc_log1p_zscore_score', 'Tmem/prolif_sc_log1p_zscore_score', 'NKL/Temra_sc_log1p_zscore_score', 'Exhaustion_sc_log1p_zscore_score', 'hs-ImCD4_sc_log1p_zscore_score', 'hs-MatCD4_sc_log1p_zscore_score', 'hs-ImCD8_sc_log1p_zscore_score', 'hs-MatCD8_sc_log1p_zscore_score', 'hs-CD8_sc_log1p_zscore_score', 'CD4+CTL_sc_log1p_zscore_score', 'CD4+PD1_sc_log1p_zscore_score', 'CD4+T_sc_log1p_zscore_score', 'CD4+Tmem_sc_log1p_zscore_score', 'CD8+T_sc_log1p_zscore_score', 'CD8+Tmem_sc_log1p_zscore_score', 'gdT_sc_log1p_zscore_score', 'CD8aa(I)_sc_log1p_zscore_score', 'CD8aa(II)_sc_log1p_zscore_score', 'NK_sc_log1p_zscore_score', 'ILC3_sc_log1p_zscore_score', 'NKT_sc_log1p_zscore_score', 'T(agonist)_sc_log1p_zscore_score', 'Tfh_sc_log1p_zscore_score', 'Th17_sc_log1p_zscore_score', 'Treg_sc_log1p_zscore_score', 'Treg(diff)_sc_log1p_zscore_score', 'union_cd4_sc_ccreg_score', 'union_cd8_sc_ccreg_score', 'cd4_sc_ccreg_score', 'cd8_sc_ccreg_score', 'Naive-/CM-like_sc_ccreg_score', 'Tem_sc_ccreg_score', 'ISG_sc_ccreg_score', 'Tmem/prolif_sc_ccreg_score', 'NKL/Temra_sc_ccreg_score', 'Exhaustion_sc_ccreg_score', 'hs-ImCD4_sc_ccreg_score', 'hs-MatCD4_sc_ccreg_score', 'hs-ImCD8_sc_ccreg_score', 'hs-MatCD8_sc_ccreg_score', 'hs-CD8_sc_ccreg_score', 'CD4+CTL_sc_ccreg_score', 'CD4+PD1_sc_ccreg_score', 'CD4+T_sc_ccreg_score', 'CD4+Tmem_sc_ccreg_score', 'CD8+T_sc_ccreg_score', 'CD8+Tmem_sc_ccreg_score', 'gdT_sc_ccreg_score', 'CD8aa(I)_sc_ccreg_score', 'CD8aa(II)_sc_ccreg_score', 'NK_sc_ccreg_score', 'ILC3_sc_ccreg_score', 'NKT_sc_ccreg_score', 'T(agonist)_sc_ccreg_score', 'Tfh_sc_ccreg_score', 'Th17_sc_ccreg_score', 'Treg_sc_ccreg_score', 'Treg(diff)_sc_ccreg_score', 'union_cd4_sc-all_log1p_score', 'union_cd8_sc-all_log1p_score', 'cd4_sc-all_log1p_score', 'cd8_sc

## CDR3 Clonotypes

In [10]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata, metric="identity", sequence="nt")
ir.tl.define_clonotypes(
    mdata,
    receptor_arms="all",
    dual_ir="primary_only",
    same_v_gene=True,
)
# allow single clones
ir.tl.clonotype_network(mdata, min_cells=1)
mdata

MuData object with n_obs × n_vars = 2401 × 36738
  3 modalities
    rna:	2401 x 36601
      obs:	'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'outliers_empty', 'outliers_multiplets', 'outliers_mito', 'outliers_ribo', 'S_score', 'G2M_score', 'phase', 'leiden_res1', 'celltypist_leiden_res1', 'celltypist_leiden_res1_conf', 'union_cd4_score', 'union_cd8_score', 'naive-cm-like_score', 'cd45ro+effector-memory_score', 'interferon-signaling_score', 'memory/proliferation_score', 'nk-like/cd45ra+effector-memory_score', 'exhaustion_score', 'cell_type', 'leidenP_res0.7', 'CD4+CTL_score', 'CD4+PD1_score', 'CD4+T_score', 'CD4+Tmem_score', 'CD8+T_score', 'CD8+Tmem_score', 'gdT_score', 'CD8aa(I)_score', 'CD8aa(II)_score', 'NK_score', 'ILC3_score', 'NKT_score', 'T(agonist)_score', 'Tfh_score', 'Th17_score', 'Treg_score', 'Treg(diff)_score', 'cell_type_biorxiv', 'cell_type_ross1', 'cell_type_ross2', 'hs-ImCD4_score', 'hs-MatCD4_score', 'hs-ImCD8_score', 'hs-MatCD8_score', 'hs-CD8_score', 'Naive-/CM-like_score', 'Tem_score', 'TTCRC4_score', 'ISG_score', 'Tmem/prolif_score', 'NKL/Temra_score', 'Exhaustion_score', 'union_cd4_sc_log1p_score', 'union_cd8_sc_log1p_score', 'cd4_sc_log1p_score', 'cd8_sc_log1p_score', 'Naive-/CM-like_sc_log1p_score', 'Tem_sc_log1p_score', 'ISG_sc_log1p_score', 'Tmem/prolif_sc_log1p_score', 'NKL/Temra_sc_log1p_score', 'Exhaustion_sc_log1p_score', 'hs-ImCD4_sc_log1p_score', 'hs-MatCD4_sc_log1p_score', 'hs-ImCD8_sc_log1p_score', 'hs-MatCD8_sc_log1p_score', 'hs-CD8_sc_log1p_score', 'CD4+CTL_sc_log1p_score', 'CD4+PD1_sc_log1p_score', 'CD4+T_sc_log1p_score', 'CD4+Tmem_sc_log1p_score', 'CD8+T_sc_log1p_score', 'CD8+Tmem_sc_log1p_score', 'gdT_sc_log1p_score', 'CD8aa(I)_sc_log1p_score', 'CD8aa(II)_sc_log1p_score', 'NK_sc_log1p_score', 'ILC3_sc_log1p_score', 'NKT_sc_log1p_score', 'T(agonist)_sc_log1p_score', 'Tfh_sc_log1p_score', 'Th17_sc_log1p_score', 'Treg_sc_log1p_score', 'Treg(diff)_sc_log1p_score', 'union_cd4_sc_log1p_zscore_score', 'union_cd8_sc_log1p_zscore_score', 'cd4_sc_log1p_zscore_score', 'cd8_sc_log1p_zscore_score', 'Naive-/CM-like_sc_log1p_zscore_score', 'Tem_sc_log1p_zscore_score', 'ISG_sc_log1p_zscore_score', 'Tmem/prolif_sc_log1p_zscore_score', 'NKL/Temra_sc_log1p_zscore_score', 'Exhaustion_sc_log1p_zscore_score', 'hs-ImCD4_sc_log1p_zscore_score', 'hs-MatCD4_sc_log1p_zscore_score', 'hs-ImCD8_sc_log1p_zscore_score', 'hs-MatCD8_sc_log1p_zscore_score', 'hs-CD8_sc_log1p_zscore_score', 'CD4+CTL_sc_log1p_zscore_score', 'CD4+PD1_sc_log1p_zscore_score', 'CD4+T_sc_log1p_zscore_score', 'CD4+Tmem_sc_log1p_zscore_score', 'CD8+T_sc_log1p_zscore_score', 'CD8+Tmem_sc_log1p_zscore_score', 'gdT_sc_log1p_zscore_score', 'CD8aa(I)_sc_log1p_zscore_score', 'CD8aa(II)_sc_log1p_zscore_score', 'NK_sc_log1p_zscore_score', 'ILC3_sc_log1p_zscore_score', 'NKT_sc_log1p_zscore_score', 'T(agonist)_sc_log1p_zscore_score', 'Tfh_sc_log1p_zscore_score', 'Th17_sc_log1p_zscore_score', 'Treg_sc_log1p_zscore_score', 'Treg(diff)_sc_log1p_zscore_score', 'union_cd4_sc_ccreg_score', 'union_cd8_sc_ccreg_score', 'cd4_sc_ccreg_score', 'cd8_sc_ccreg_score', 'Naive-/CM-like_sc_ccreg_score', 'Tem_sc_ccreg_score', 'ISG_sc_ccreg_score', 'Tmem/prolif_sc_ccreg_score', 'NKL/Temra_sc_ccreg_score', 'Exhaustion_sc_ccreg_score', 'hs-ImCD4_sc_ccreg_score', 'hs-MatCD4_sc_ccreg_score', 'hs-ImCD8_sc_ccreg_score', 'hs-MatCD8_sc_ccreg_score', 'hs-CD8_sc_ccreg_score', 'CD4+CTL_sc_ccreg_score', 'CD4+PD1_sc_ccreg_score', 'CD4+T_sc_ccreg_score', 'CD4+Tmem_sc_ccreg_score', 'CD8+T_sc_ccreg_score', 'CD8+Tmem_sc_ccreg_score', 'gdT_sc_ccreg_score', 'CD8aa(I)_sc_ccreg_score', 'CD8aa(II)_sc_ccreg_score', 'NK_sc_ccreg_score', 'ILC3_sc_ccreg_score', 'NKT_sc_ccreg_score', 'T(agonist)_sc_ccreg_score', 'Tfh_sc_ccreg_score', 'Th17_sc_ccreg_score', 'Treg_sc_ccreg_score', 'Treg(diff)_sc_ccreg_score', 'union_cd4_sc-all_log1p_score', 'union_cd8_sc-all_log1p_score', 'cd4_sc-all_log1p_score', 'cd8_sc

In [11]:
df_vdj = ir.get.airr(
    mdata, ["junction_aa", "v_call", "d_call", "j_call"], ["VJ_1", "VDJ_1"]
)
# Ensure the indices align between df_vdj and mdata["airr"].obs
# Then add the columns from df_vdj to mdata["airr"].obs
mdata["airr"].obs = mdata["airr"].obs.join(df_vdj, how="left")

### Add cell type annotations to TCR adata

In [12]:
# add cell type annotations to airr
mdata["airr"].obs["cd4_cd8_subset_thresh3"] = (
    mdata["prot"].obs["cd4_cd8_subset_thresh3"].astype("category")
)
mdata["airr"].obs["cell_type_ross2"] = (
    mdata["prot"].obs["cell_type_ross2"].astype("category")
)

In [13]:
mdata["airr"].obs

,receptor_type,receptor_subtype,chain_pairing,clone_id,clone_id_size,VJ_1_junction_aa,VJ_1_v_call,VJ_1_d_call,VJ_1_j_call,VDJ_1_junction_aa,VDJ_1_v_call,VDJ_1_d_call,VDJ_1_j_call,cd4_cd8_subset_thresh3,cell_type_ross2
cell_id,,,,,,,,,,,,,,,
AAACCTGAGTACGACG-1,TCR,TRA+TRB,extra VJ,0,42,CAASVTGTASKLTF,TRAV29/DV5,None,TRAJ44,CASSEDSSYEQYF,TRBV25-1,None,TRBJ2-7,DP,EM
AAACCTGGTAAACACA-1,TCR,TRA+TRB,single pair,1,9,CAGGGSQGNLIF,TRAV27,None,TRAJ42,CASSLDNSYEQYF,TRBV12-4,None,TRBJ2-7,DN,TM
AAACCTGGTCAAGCGA-1,TCR,TRA+TRB,extra VDJ,2,1,CALRISSGSARQLTF,TRAV19,None,TRAJ22,CASRQGSYEQYF,TRBV10-1,TRBD1,TRBJ2-7,DN,TM
AAACCTGGTGGTCCGT-1,TCR,TRA+TRB,single pair,3,102,CAAIGNEKLTF,TRAV29/DV5,None,TRAJ48,CASSVGQGETQYF,TRBV9,TRBD1,TRBJ2-5,8SP,Mem(P)
AAACCTGTCAGCTTAG-1,TCR,TRA+TRB,single pair,4,3,CAGEETSGSRLTF,TRAV27,None,TRAJ58,CASSLQGETQYF,TRBV27,None,TRBJ2-5,DN,Mem(P)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACACTCGACG-1,TCR,TRA+TRB,extra VJ,58,8,CAMSEETSGSRLTF,TRAV12-3,None,TRAJ58,CAWRGHTGELFF,TRBV30,None,TRBJ2-2,8SP,EM
TTTGTCACAGGAATGC-1,TCR,TRA+TRB,single pair,29,29,CADRGSTLGRLYF,TRAV3,None,TRAJ18,CASSYRDNTDTQYF,TRBV6-5,None,TRBJ2-3,8SP,Eff/Ex
TTTGTCACAGGGTTAG-1,TCR,TRA+TRB,single pair,355,8,CATDSGGSNYKLTF,TRAV17,None,TRAJ53,CASSPPDTYEQYF,TRBV18,None,TRBJ2-7,DN,Mem(P)


### Export TCR sequences with specific metadata

In [14]:
mdata["airr"].obs.groupby(
    [
        "clone_id",
        "cd4_cd8_subset_thresh3",
        "cell_type_ross2",
        "VJ_1_junction_aa",
        "VJ_1_v_call",
        "VJ_1_j_call",
        "VDJ_1_junction_aa",
        "VDJ_1_v_call",
        "VDJ_1_j_call",
    ],
    observed=True,
    dropna=False,
).size().reset_index(name="count").to_csv(PATH_EXPORT_DIR / f"airr_grouped_counts.csv")

In [ ]:
# export tcr mdata
## NEED TO REMOVE "VJ_1_d_call" from tcr obs for saving (sometimes columns can be buggy!!)
del mdata["airr"].obs["VJ_1_d_call"]
mdata.write_h5mu(PATH_DATA_DIR / f"mdata_analyze_tcr_intersected.h5mu")
mdata

/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/divy/miniconda3/envs/sc-dense/lib/python3.11/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 2401 × 36738
  3 modalities
    rna:	2401 x 36601
      obs:	'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'outliers_empty', 'outliers_multiplets', 'outliers_mito', 'outliers_ribo', 'S_score', 'G2M_score', 'phase', 'leiden_res1', 'celltypist_leiden_res1', 'celltypist_leiden_res1_conf', 'union_cd4_score', 'union_cd8_score', 'naive-cm-like_score', 'cd45ro+effector-memory_score', 'interferon-signaling_score', 'memory/proliferation_score', 'nk-like/cd45ra+effector-memory_score', 'exhaustion_score', 'cell_type', 'leidenP_res0.7', 'CD4+CTL_score', 'CD4+PD1_score', 'CD4+T_score', 'CD4+Tmem_score', 'CD8+T_score', 'CD8+Tmem_score', 'gdT_score', 'CD8aa(I)_score', 'CD8aa(II)_score', 'NK_score', 'ILC3_score', 'NKT_score', 'T(agonist)_score', 'Tfh_score', 'Th17_score', 'Treg_score', 'Treg(diff)_score', 'cell_type_biorxiv', 'cell_type_ross1', 'cell_type_ross2', 'hs-ImCD4_score', 'hs-MatCD4_score', 'hs-ImCD8_score', 'hs-MatCD8_score', 'hs-CD8_score', 'Naive-/CM-like_score', 'Tem_score', 'TTCRC4_score', 'ISG_score', 'Tmem/prolif_score', 'NKL/Temra_score', 'Exhaustion_score', 'union_cd4_sc_log1p_score', 'union_cd8_sc_log1p_score', 'cd4_sc_log1p_score', 'cd8_sc_log1p_score', 'Naive-/CM-like_sc_log1p_score', 'Tem_sc_log1p_score', 'ISG_sc_log1p_score', 'Tmem/prolif_sc_log1p_score', 'NKL/Temra_sc_log1p_score', 'Exhaustion_sc_log1p_score', 'hs-ImCD4_sc_log1p_score', 'hs-MatCD4_sc_log1p_score', 'hs-ImCD8_sc_log1p_score', 'hs-MatCD8_sc_log1p_score', 'hs-CD8_sc_log1p_score', 'CD4+CTL_sc_log1p_score', 'CD4+PD1_sc_log1p_score', 'CD4+T_sc_log1p_score', 'CD4+Tmem_sc_log1p_score', 'CD8+T_sc_log1p_score', 'CD8+Tmem_sc_log1p_score', 'gdT_sc_log1p_score', 'CD8aa(I)_sc_log1p_score', 'CD8aa(II)_sc_log1p_score', 'NK_sc_log1p_score', 'ILC3_sc_log1p_score', 'NKT_sc_log1p_score', 'T(agonist)_sc_log1p_score', 'Tfh_sc_log1p_score', 'Th17_sc_log1p_score', 'Treg_sc_log1p_score', 'Treg(diff)_sc_log1p_score', 'union_cd4_sc_log1p_zscore_score', 'union_cd8_sc_log1p_zscore_score', 'cd4_sc_log1p_zscore_score', 'cd8_sc_log1p_zscore_score', 'Naive-/CM-like_sc_log1p_zscore_score', 'Tem_sc_log1p_zscore_score', 'ISG_sc_log1p_zscore_score', 'Tmem/prolif_sc_log1p_zscore_score', 'NKL/Temra_sc_log1p_zscore_score', 'Exhaustion_sc_log1p_zscore_score', 'hs-ImCD4_sc_log1p_zscore_score', 'hs-MatCD4_sc_log1p_zscore_score', 'hs-ImCD8_sc_log1p_zscore_score', 'hs-MatCD8_sc_log1p_zscore_score', 'hs-CD8_sc_log1p_zscore_score', 'CD4+CTL_sc_log1p_zscore_score', 'CD4+PD1_sc_log1p_zscore_score', 'CD4+T_sc_log1p_zscore_score', 'CD4+Tmem_sc_log1p_zscore_score', 'CD8+T_sc_log1p_zscore_score', 'CD8+Tmem_sc_log1p_zscore_score', 'gdT_sc_log1p_zscore_score', 'CD8aa(I)_sc_log1p_zscore_score', 'CD8aa(II)_sc_log1p_zscore_score', 'NK_sc_log1p_zscore_score', 'ILC3_sc_log1p_zscore_score', 'NKT_sc_log1p_zscore_score', 'T(agonist)_sc_log1p_zscore_score', 'Tfh_sc_log1p_zscore_score', 'Th17_sc_log1p_zscore_score', 'Treg_sc_log1p_zscore_score', 'Treg(diff)_sc_log1p_zscore_score', 'union_cd4_sc_ccreg_score', 'union_cd8_sc_ccreg_score', 'cd4_sc_ccreg_score', 'cd8_sc_ccreg_score', 'Naive-/CM-like_sc_ccreg_score', 'Tem_sc_ccreg_score', 'ISG_sc_ccreg_score', 'Tmem/prolif_sc_ccreg_score', 'NKL/Temra_sc_ccreg_score', 'Exhaustion_sc_ccreg_score', 'hs-ImCD4_sc_ccreg_score', 'hs-MatCD4_sc_ccreg_score', 'hs-ImCD8_sc_ccreg_score', 'hs-MatCD8_sc_ccreg_score', 'hs-CD8_sc_ccreg_score', 'CD4+CTL_sc_ccreg_score', 'CD4+PD1_sc_ccreg_score', 'CD4+T_sc_ccreg_score', 'CD4+Tmem_sc_ccreg_score', 'CD8+T_sc_ccreg_score', 'CD8+Tmem_sc_ccreg_score', 'gdT_sc_ccreg_score', 'CD8aa(I)_sc_ccreg_score', 'CD8aa(II)_sc_ccreg_score', 'NK_sc_ccreg_score', 'ILC3_sc_ccreg_score', 'NKT_sc_ccreg_score', 'T(agonist)_sc_ccreg_score', 'Tfh_sc_ccreg_score', 'Th17_sc_ccreg_score', 'Treg_sc_ccreg_score', 'Treg(diff)_sc_ccreg_score', 'union_cd4_sc-all_log1p_score', 'union_cd8_sc-all_log1p_score', 'cd4_sc-all_log1p_score', 'cd8_sc